# Training Pipeline

We choose LSTM as a classification model 


### Imports

In [8]:
import os

import joblib
import numpy as np
import pandas as pd
import hopsworks
from datasets import load_dataset
from hsml.model_schema import ModelSchema
from hsml.schema import Schema
from huggingface_hub import notebook_login
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib as plt
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.models import Sequential

### Connect to Hopsworks, Huggingface & WandB

In [2]:
project = hopsworks.login() 

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5322


Connected. Call `.close()` to terminate connection gracefully.


In [3]:
notebook_login()

## Load Data from Huggingface

In [6]:
ds = load_dataset("eengel7/sentiment_analysis_training", split='train')
data_df = pd.DataFrame(data = ds, columns=['Sentiment',  'Headline'])
y = pd.get_dummies(data_df['Sentiment'])

X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(data_df['Headline'].to_list()), y, test_size=0.2, random_state=42)

2023-01-13 17:22:43,793 WARNING: Using custom data configuration eengel7--sentiment_analysis_training-0b9cd88a291c773d
2023-01-13 17:22:44,011 WARNING: Found cached dataset parquet (/Users/evaengel/.cache/huggingface/datasets/eengel7___parquet/eengel7--sentiment_analysis_training-0b9cd88a291c773d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [7]:
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

## LSTM Model

In [49]:
# parameter initialization -  arbitrarily choosen
voc_size = 5000 
max_len = 60
embedding_vector_features = 40

In [58]:
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = max_len))
model.add(Dropout(0.5))
model.add(LSTM(200))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [63]:
# Train model
batch_size = 64
history = model.fit(X_train, y_train,
                      validation_data=(X_test, y_test),
                      batch_size=batch_size, epochs= 50, verbose=1)

Epoch 1/50
144/144 [==============================] - 20s 135ms/step - loss: 0.4085 - accuracy: 0.8531 - val_loss: 1.0227 - val_accuracy: 0.6378
Epoch 2/50
144/144 [==============================] - 22s 150ms/step - loss: 0.3935 - accuracy: 0.8567 - val_loss: 1.0086 - val_accuracy: 0.6313
Epoch 3/50
144/144 [==============================] - 20s 139ms/step - loss: 0.3623 - accuracy: 0.8727 - val_loss: 1.1753 - val_accuracy: 0.6309
Epoch 4/50
144/144 [==============================] - 19s 131ms/step - loss: 0.3516 - accuracy: 0.8731 - val_loss: 1.2127 - val_accuracy: 0.6296
Epoch 5/50
144/144 [==============================] - 19s 131ms/step - loss: 0.3474 - accuracy: 0.8746 - val_loss: 1.0443 - val_accuracy: 0.6383
Epoch 6/50
144/144 [==============================] - 19s 132ms/step - loss: 0.3188 - accuracy: 0.8885 - val_loss: 1.0600 - val_accuracy: 0.6378
Epoch 7/50
144/144 [==============================] - 22s 153ms/step - loss: 0.3205 - accuracy: 0.8877 - val_loss: 1.0250 - val_ac

In [10]:
def plot_training_hist(history):
    '''Function to plot history for accuracy and loss'''
    
    fig, ax = plt.subplots(1, 2, figsize=(10,4))
    # first plot
    ax[0].plot(history.history['accuracy'])
    ax[0].plot(history.history['val_accuracy'])
    ax[0].set_title('Model Accuracy')
    ax[0].set_xlabel('epoch')
    ax[0].set_ylabel('accuracy')
    ax[0].legend(['train', 'validation'], loc='best')
    # second plot
    ax[1].plot(history.history['loss'])
    ax[1].plot(history.history['val_loss'])
    ax[1].set_title('Model Loss')
    ax[1].set_xlabel('epoch')
    ax[1].set_ylabel('loss')
    ax[1].legend(['train', 'validation'], loc='best')
def plot_training_hist(history):
    '''Function to plot history for accuracy and loss'''
    
    fig, ax = plt.subplots(1, 2, figsize=(10,4))
    # first plot
    ax[0].plot(history.history['accuracy'])
    ax[0].plot(history.history['val_accuracy'])
    ax[0].set_title('Model Accuracy')
    ax[0].set_xlabel('epoch')
    ax[0].set_ylabel('accuracy')
    ax[0].legend(['train', 'validation'], loc='best')
    # second plot
    ax[1].plot(history.history['loss'])
    ax[1].plot(history.history['val_loss'])
    ax[1].set_title('Model Loss')
    ax[1].set_xlabel('epoch')
    ax[1].set_ylabel('loss')
    ax[1].legend(['train', 'validation'], loc='best')

print(model.summary())    
plot_training_hist(history)

NameError: name 'history' is not defined

## Upload the model

In [64]:
mr = project.get_model_registry()

model_dir="headlines_sentiment_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(model, model_dir + "/headlines_sentiment_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dropout
.........vars
......dropout_1
.........vars
......embedding
.........vars
............0
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-01-13 17:01:14         2508
metadata.json                                  2023-01-13 17:01:14           64
variables.h5        

['headlines_sentiment_model/headlines_sentiment_model.pkl']

In [65]:
history1 = history
acc = history1.history['accuracy'][-1]

headlines_sentiment_model = mr.python.create_model(
        name = "headlines_sentiment_model", 
        metrics = {"accuracy": acc},
        model_schema=model_schema,
        description="Predicting Sentiment of Headlines"
    )

In [66]:
headlines_sentiment_model.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/5322/models/headlines_sentiment_model/4


Model(name: 'headlines_sentiment_model', version: 4)